## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_2 = pd.read_csv("../../바이너리_최종데이터_1004/gcc8/o2/o2_bincore8.csv", index_col=0)
print(bin8_2.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_2.reset_index(inplace=True, drop=True)

print('shape -', bin8_2.shape)
print('reset_index 완료')
print('input data shape')
bin8_2.head()

(28744700, 2)
shape - (28744700, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_2['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_2['label'].value_counts())

256
0    28706035
1       38665
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin8_2[bin8_2['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_2), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_2', len(ls_idx_bin))

# loc 로 수정필요
bin8_2_Ngram = bin8_2.loc[ls_idx_bin,:].copy()

1237280
나머지 0
최종 길이 1237280
bin8_2 1237280


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_2_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_2.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_2)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_2[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

38665.0
0 13293519
1000 800224
2000 25174221
3000 359537
4000 7142748
5000 21232651
6000 27461404
7000 22697614
8000 22565176
9000 5110123
10000 27419453
11000 21509015
12000 4819624
13000 25713622
14000 17843023
15000 2339151
16000 21636542
17000 28246601
18000 7070957
19000 18927471
20000 15842946
21000 27220502
22000 829611
23000 23261120
24000 25865726
25000 23091101
26000 4310968
27000 1561335
28000 15307483
29000 20544474
30000 8749070
31000 27328753
32000 24988068
33000 25433291
34000 15742807
35000 17108130
36000 1359566
37000 8938540
38000 27880417
완료
38665


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_2['label'].value_counts()[1])

38665
38665


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_2_Ngram])
final.shape

(2474560, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_2_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_2_onehot_Ngram = pd.concat([final['label'], bc8_2_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_2_onehot_Ngram.shape)

원핫인코딩완료
(2474560, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc8_2 = bc8_2_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_2 = bc8_2_onehot_Ngram['label'].to_numpy()
print(x_bc8_2.shape, x_bc8_2.shape)

x_bc8_2 = x_bc8_2.reshape(-1, right_idx, x_bc8_2.shape[1])
y_bc8_2 = y_bc8_2.reshape(-1, right_idx, 1)

print(x_bc8_2.shape, y_bc8_2.shape)

(2474560, 256) (2474560, 256)
(77330, 32, 256) (77330, 32, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_2.shape[0])

x_bc8_2 = x_bc8_2[p]
y_bc8_2 = y_bc8_2[p]

print(x_bc8_2.shape, y_bc8_2.shape)

(77330, 32, 256) (77330, 32, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc8_2, y_bc8_2):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc8_2[train],
               y_bc8_2[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc8_2[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc8_2[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69597 samples
Epoch 1/10
69597/69597 [==============================] - 104s 1ms/sample - loss: 0.0141 - accuracy: 0.9960
Epoch 2/10
69597/69597 [==============================] - 47s 680us/sample - loss: 0.0025 - accuracy: 0.9993
Epoch 3/10
69597/69597 [==============================] - 47s 676us/sample - loss: 0.0018 - accuracy: 0.9995
Epoch 4/1

accuracy_score 0.9997696560196561
recall_score 0.9927216585796207
precision_score 0.9946961325966851
f1_score 0.9937079147808808
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69597 samples
Epoch 1/10
69597/69597 [==============================] - 82s 1ms/sample - loss: 5.9928e-04 - accuracy: 0.9998
Epoch 2/10
69597/69597 [==============================] - 43s 625us/sample - loss: 5.0815e-04

accuracy_score 0.9998626018362861
recall_score 0.9954819277108434
precision_score 0.997198275862069
f1_score 0.9963393626184325
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69597 samples
Epoch 1/10
69597/69597 [==============================] - 80s 1ms/sample - loss: 2.5154e-04 - accuracy: 0.9999
Epoch 2/10
69597/69597 [==============================] - 33s 474us/sample - loss: 2.1843e-04 

accuracy_score 0.9998908896935212
recall_score 0.9971258014592085
precision_score 0.9969053934571176
f1_score 0.9970155852768874
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69597 samples
Epoch 1/10
69597/69597 [==============================] - 89s 1ms/sample - loss: 1.0798e-04 - accuracy: 1.0000
Epoch 2/10
69597/69597 [==============================] - 46s 668us/sample - loss: 4.4844e-05

accuracy_score 0.999951506530454
recall_score 0.9982650184341791
precision_score 0.9991317560234426
f1_score 0.9986981991755263
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69597 samples
Epoch 1/10
69597/69597 [==============================] - 80s 1ms/sample - loss: 5.8288e-05 - accuracy: 1.0000
Epoch 2/10
69597/69597 [==============================] - 46s 665us/sample - loss: 3.0783e-05 

...

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69597 samples
Epoch 1/10
69597/69597 [==============================] - 66s 950us/sample - loss: 3.3561e-06 - accuracy: 1.0000
Epoch 2/10
69597/69597 [==============================] - 46s 655us/sample - loss: 2.2881e-06 - accuracy: 1.0000
Epoch 3/10
69597/69597 [==============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69597 samples
Epoch 1/10
69597/69597 [==============================] - 55s 793us/sample - loss: 6.2135e-09 - accuracy: 1.0000
Epoch 2/10
69597/69597 [==============================] - 31s 439us/sample - loss: 1.1424e-06 - accuracy: 1.0000
Epoch 3/10
69597/69597 [==============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69597 samples
Epoch 1/10
69597/69597 [==============================] - 42s 602us/sample - loss: 1.9190e-06 - accuracy: 1.0000
Epoch 2/10
69597/69597 [==============================] - 30s 430us/sample - loss: 8.1591e-10 - accuracy: 1.0000
Epoch 3/10
69597/69597 [==============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69597 samples
Epoch 1/10
69597/69597 [==============================] - 42s 602us/sample - loss: 3.8361e-10 - accuracy: 1.0000
Epoch 2/10
69597/69597 [==============================] - 30s 430us/sample - loss: 3.2060e-10 - accuracy: 1.0000
Epoch 3/10
69597/69597 [==============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9997696560196561, 0.9998626018362861, 0.9998908896935212, 0.999951506530454, 0.9999959588775378, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Recall: [0.9927216585796207, 0.9954819277108434, 0.9971258014592085, 0.9982650184341791, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9946961325966851, 0.997198275862069, 0.9969053934571176, 0.9991317560234426, 0.9997825141365811, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9937079147808808, 0.9963393626184325, 0.9970155852768874, 0.9986981991755263, 0.9998912452419794, 1.0, 1.0, 1.0, 1.0, 1.0]


## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999470612957454
10-Fold Cross_validation. Recall : 0.9983594406183851
10-Fold Cross_validation. Precision : 0.9987714072075896
10-Fold Cross_validation. F1-Score : 0.9985652307093706


In [13]:
model1.save('gcc8_bin_core_s32_h48_o2.h5')
print('save 완료')

save 완료
